In [1]:
import sys
sys.path.insert(1, '/home/gyzhang/merlin/src')
# import matplotlib.pyplot as plt
# %matplotlib inline
import bandmat.linalg as bla
import numpy as np
import bandmat as bm
from frontend.acoustic_composition import AcousticComposition
ac = AcousticComposition()

In [2]:
frames=20

In [23]:
test_static = np.random.rand(20)
test_static = np.reshape(test_static,(20,1))
delta_features = ac.compute_dynamic_matrix(test_static, [-0.5, 0.0, 0.5], 20,1)
acc_features = ac.compute_dynamic_matrix(test_static, [1, -2.0, 1], 20,1)
merge = np.concatenate((test_static, delta_features, acc_features),axis=1)
mean_test_d = np.mean(merge, axis=0)
std_test_d = np.std(merge,axis=0)
var_test_d = std_test_d**2

In [24]:
var_test_d = np.tile(var_test_d, (frames,1))

In [25]:
var_test_d[0, 1] = 100000000000
var_test_d[0, 2] = 100000000000
var_test_d[20 - 1, 1] = 100000000000
var_test_d[20 - 1, 2] = 100000000000

In [26]:
b_frames = merge / var_test_d
tau_frames = 1.0 / var_test_d

In [27]:
windows = [
            (0, 0, np.array([1.0])),
            (1, 1, np.array([-0.5, 0.0, 0.5])),
            (1, 1, np.array([1.0, -2.0, 1.0])),
        ]

In [28]:
win_mats = []
for l, u, win_coeff in windows:
    assert l >= 0 and u >= 0
    assert len(win_coeff) == l + u + 1
    win_coeffs = np.tile(np.reshape(win_coeff, (l + u + 1, 1)), 20)
    win_mat = bm.band_c_bm(u, l, win_coeffs).T
    win_mats.append(win_mat)


In [29]:
windows = np.zeros((60, 20))

In [30]:
row = 0
for win1,win2,win3 in zip(win_mats[0].full(),win_mats[1].full(),win_mats[2].full()):
    windows[row,:] = win1
    windows[row+1,:] = win1
    windows[row+2,:] = win1
    row = row+3

In [31]:
var_pre = np.reshape(tau_frames,(1,-1))

In [32]:
var_pre_diag = np.diag(var_pre[0])

In [33]:
windows_up = np.matmul(np.matmul(windows.T, var_pre_diag),windows)

In [34]:
b = np.matmul(windows.T, np.reshape(b_frames,(-1,1)))

In [16]:
x = np.linalg.solve(windows_up,b)


In [36]:
b

array([[ 8.02093494],
       [ 1.85672073],
       [ 2.94902282],
       [15.89347204],
       [13.44529705],
       [ 7.80879742],
       [-2.48528666],
       [ 3.28556595],
       [ 4.3468617 ],
       [ 0.60037506],
       [10.61792009],
       [ 0.8839295 ],
       [ 6.15598225],
       [10.90576722],
       [-2.00715594],
       [-2.46371141],
       [ 9.83143006],
       [ 8.75790517],
       [-3.28890503],
       [ 1.21101024]])

In [22]:
windows_up

array([[38.12444765,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , 38.12444765,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , 38.12444765,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , 38.12444765,  0.        ,
         0.        ,  0.        ,  0.        ,  

In [18]:
test_static

array([[0.32478961],
       [0.05386585],
       [0.66008477],
       [0.37814473],
       [0.56376993],
       [0.75632645],
       [0.06813711],
       [0.26565781],
       [0.82723628],
       [0.28552135],
       [0.01662472],
       [0.65924215],
       [0.05673126],
       [0.28699929],
       [0.77845354],
       [0.44229557],
       [0.27063136],
       [0.56394635],
       [0.6272156 ],
       [0.13072541]])

In [ ]:
sdw = max([win_mat.l + win_mat.u for win_mat in win_mats])
b = np.zeros((frames,))
prec = bm.zeros(sdw, sdw, 20)
for win_index, win_mat in enumerate(win_mats):
            bm.dot_mv_plus_equals(win_mat.T, b_frames[:, win_index], target=b)
            bm.dot_mm_plus_equals(win_mat.T, win_mat, target_bm=prec,diag=np.float64(tau_frames[:, win_index]))

In [ ]:
mean_traj = bla.solveh(prec, b)

In [ ]:
mean_traj

In [ ]:
test_static

In [ ]:
a = np.array([[1,2,3],[2,3,4],[5,6,7]])

In [ ]:
np.reshape(a,(-1,1))